## This Notebook is a combination of everything needed to run our cook vision on a machine

### Run this to install all the requirements if you don't hava them already installed in your venv

In [ ]:
%pip install -r requirements.txt

#### Modules needed for various runs

In [ ]:
import os
import shutil
from PIL import Image
import json
from collections import defaultdict, Counter
from fuzzywuzzy import fuzz
import streamlit as st
import tempfile
from ultralytics import YOLO
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


### After downloading the FOOD-101 Data, you should change the DIR as needed to the root DIR of the food data and align all the other DIR as needed

#### Script for splitting the food dataset

In [ ]:
# === Configuration ===
raw_images_root = "/home/classes/ee7722/ee772210/Downloads/food-101/images/"  # path to original class folders (e.g. apple_pie/*.jpg)
output_root = "datasets/food101_yolo"
train_txt = "/home/classes/ee7722/ee772210/Downloads/food-101/meta/train.txt"
val_txt = "/home/classes/ee7722/ee772210/Downloads/food-101/meta/test.txt"  # Food-101 calls it test.txt

# === Output directories ===
image_train = os.path.join(output_root, "images", "train")
image_val = os.path.join(output_root, "images", "val")
label_train = os.path.join(output_root, "labels", "train")
label_val = os.path.join(output_root, "labels", "val")

# === Create dirs if they don't exist ===
for d in [image_train, image_val, label_train, label_val]:
    os.makedirs(d, exist_ok=True)

# === Load class names ===
with open("/home/classes/ee7722/ee772210/Downloads/food-101/meta/classes.txt") as f:
    class_names = [line.strip() for line in f]
class_dict = {name: idx for idx, name in enumerate(class_names)}

# === Function to convert image and generate label ===
def process_list(txt_file, image_dir, label_dir):
    with open(txt_file, "r") as f:
        for line in f:
            rel_path = line.strip()  # e.g., apple_pie/123456
            class_name = rel_path.split("/")[0]
            img_name = rel_path.split("/")[1] + ".jpg"
            class_id = class_dict[class_name]

            src_img_path = os.path.join(raw_images_root, class_name, img_name)
            dst_img_path = os.path.join(image_dir, f"{class_name}_{img_name}")
            dst_lbl_path = os.path.join(label_dir, f"{class_name}_{img_name.replace('.jpg', '.txt')}")

            if not os.path.exists(src_img_path):
                print(f"Image missing: {src_img_path}")
                continue

            shutil.copy2(src_img_path, dst_img_path)

            # Create a bounding box that spans most of the image (assume object-centered)
            try:
                with Image.open(src_img_path) as img:
                    w, h = img.size
                # YOLO format: <class> <x_center> <y_center> <width> <height> (normalized)
                bbox = [class_id, 0.5, 0.5, 0.9, 0.9]
                with open(dst_lbl_path, "w") as f_lbl:
                    f_lbl.write(" ".join([str(x) for x in bbox]) + "\n")
            except Exception as e:
                print(f"Failed processing {src_img_path}: {e}")

# === Generate train and val splits with labels ===
process_list(train_txt, image_train, label_train)
process_list(val_txt, image_val, label_val)

print("✅ Dataset organized and YOLO labels generated.")

### Using the data.yaml file for Yolov8 modelling

In [ ]:
# Adjust Epoch as needed
%yolo task=detect      mode=train      model=yolov8n.pt      data=data.yaml      epochs=50      imgsz=640      batch=16

### Generating dish to ingredients json file

In [ ]:
# === CONFIGURATION ===
CLASSES_FILE = "/home/classes/ee7722/ee772210/Downloads/food-101/meta/classes.txt"
RECIPES_FILE = "datasets/recipe-ingredients/train.json"
OUTPUT_FILE = "dish2ingredients.json"

FUZZY_THRESHOLD = 80  # Match quality (0–100); lower = more aggressive

# === STEP 1: Load Food-101 class names ===
with open(CLASSES_FILE, "r") as f:
    food101_dishes = [line.strip() for line in f]

# === STEP 2: Load Kaggle recipe dataset ===
with open(RECIPES_FILE, "r") as f:
    recipes = json.load(f)

# === STEP 3: Match dishes to recipes by fuzzy keyword matching ===
dish_to_ingredients = defaultdict(list)

for dish in food101_dishes:
    dish_name = dish.replace("_", " ").lower()

    for recipe in recipes:
        ingredients = [ing.lower() for ing in recipe["ingredients"]]
        combined = " ".join(ingredients)

        # Fuzzy match the dish name to the combined ingredients string
        score = fuzz.partial_ratio(dish_name, combined)
        if score >= FUZZY_THRESHOLD:
            dish_to_ingredients[dish].extend(ingredients)

print(f"✅ Matched recipes for {len(dish_to_ingredients)} out of {len(food101_dishes)} classes.")

# === STEP 4: Count and save top ingredients for each dish ===
dish2ingredients_final = {}
for dish, all_ingredients in dish_to_ingredients.items():
    ingredient_counts = Counter(all_ingredients)
    top_ingredients = [ing for ing, _ in ingredient_counts.most_common(10)]
    dish2ingredients_final[dish] = top_ingredients

# Save to JSON
with open(OUTPUT_FILE, "w") as f:
    json.dump(dish2ingredients_final, f, indent=2)

print(f"✅ Saved dish2ingredients.json to: {OUTPUT_FILE}")

## Using Streamlit UI to give a better UI Representation

In [ ]:
# Load model and dish-to-ingredient map
model = YOLO("runs/detect/train/weights/best.pt")

with open("dish2ingredients.json") as f:
    dish_map = json.load(f)


# === Load Mistral-7B-Instruct model locally ===
@st.cache_resource
def load_mistral_model():
    model_id = "mistralai/Mistral-7B-Instruct-v0.1"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    return tokenizer, model

tokenizer, mistral_model = load_mistral_model()


# === Step 2: Recipe Generator ===
def generate_recipe_steps(dish, ingredients):
    prompt = f"Give me a clear at least 5-step recipe for making {dish} using the following ingredients: {', '.join(ingredients)}"
    inputs = tokenizer(prompt, return_tensors="pt").to(mistral_model.device)
    outputs = mistral_model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# === Streamlit UI ===
st.set_page_config(page_title="CookVision", layout="centered")
st.title("🍳 CookVision: AI Cooking Assistant")
st.markdown("Upload a food image to detect the dish and get a recipe with likely ingredients.")

upload_dir = "data/uploads"
os.makedirs(upload_dir, exist_ok=True)

uploaded_file = st.file_uploader("📷 Upload a food image", type=["jpg", "png", "jpeg"])

if uploaded_file:
    with tempfile.NamedTemporaryFile(delete=False, dir=upload_dir, suffix=".jpg") as tmp:
        tmp.write(uploaded_file.read())
        image_path = tmp.name

    image = Image.open(image_path)
    st.image(image, caption="📸 Uploaded Image", use_container_width=True)

    # === Detection Phase ===
    with st.spinner("Detecting dish..."):
        results = model(image_path)
        detected_dishes = set()

        for r in results:
            for box in r.boxes:
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                detected_dishes.add(class_name)

    if detected_dishes:
        for dish in detected_dishes:
            st.subheader(f"🍽️ Detected Dish: {dish.capitalize()}")

            # Get ingredients
            ingredients = dish_map.get(dish, ["❓ Ingredients not found"])
            st.markdown("**🧾 Inferred Ingredients:** " + ", ".join(ingredients))

            # Get recipe steps
            with st.spinner("🧠 Generating recipe..."):
                recipe = generate_recipe_steps(dish, ingredients)

            st.markdown("**👩‍🍳 Suggested Recipe Steps:**")
            st.markdown(recipe)
    else:
        st.warning("⚠️ No recognizable dish detected.")
else:
    st.info("Upload a food photo to get started!")

In [ ]:
# To run the Streamlit UI
%streamlit runs app_ui2.py